# Title
Collaborators: Albert Chen, Alex Chen, Aaron Lin, Srujan Penikelapati

# Introduction

Faculty are the lifeblood of any university. They conduct cutting-edge research to discover new insights that push the boundary of the field they are studying. They also teach classes, helping students learn new skills and knowledge that will benefit them in their professional lives. As in any occupation, professors vary in the quality of their research and their teaching ability, which directly impacts the quality of education that students who take their classes receive.

Students understand the fact that their choice of professor for a course can be the difference between an engaging, informative semester or a less than pleasing experience. Prior to registration, students often check professor reviews, online posts regarding the class, grade distributions, and other sources of information to determine whose class to choose. 

The goal of this project is to see the connection between how effective a professor is and possible determining factors of their teaching ability. Using data science, we will review salary data, years of experience, and average student rating of professors here at the University of Maryland to see how they correlate with their average Grade Point Average (GPA) for students across their classes. For example, does a professor with a high salary and many years of experience at UMD reflect in higher student GPAs? Once we build a correlative model, we can also pose performative questions in reverse, like whether the University is getting the best 'bang-for-their-buck' given a professor's salary and their average GPA as compared to other professors in the same field. 








# Data Collection

We first need to collect data that is relevant to our study. To get grade distributions and student reviews, we made GET requests to the [PlanetTerp API](https://planetterp.com/api/) and stored the data in dataframes. To get information on a professor's salaries and how long the've been teaching, we made GET requests to the [Diamondback API](https://api.dbknews.com/docs/#/salary) and also stored that data in a dataframe. However, performing all of these requests can take a long time, making it impractical to work efficiently. As such, we stored our dataframes into various .csv files. This also means that our data is accurate up to April 28th, 2023, as from there on we read from our csv files instead of requesting from the websites themselves. The code used for that is below.